# 00-5: ComfyUI Local - Test Rapide

**Objectif**: Valider connexion et génération basique ComfyUI local

**Prérequis**:
- ComfyUI démarré sur port 8188
- Modèle Qwen-Image-Edit-2509-FP8 chargé
- Custom nodes Qwen installés

**Durée**: ~2-3 minutes

## Cell 1: Import et Configuration

In [ ]:
# Imports
import sys
import os

# Ajouter shared helpers au path
# Note: Papermill exécute depuis le répertoire du notebook, pas depuis la racine
# On utilise donc un chemin absolu pour garantir la compatibilité
shared_path = r'd:\Dev\CoursIA\MyIA.AI.Notebooks\GenAI\shared'
if shared_path not in sys.path:
    sys.path.insert(0, shared_path)

from helpers.comfyui_client import create_client

print("✅ Imports réussis")

## Cell 2: Test Connexion ComfyUI

In [ ]:
# Test connexion rapide via create_client()
print("🔍 Test connexion ComfyUI...")

try:
    test_client = create_client()
    
    # Test basique avec /system_stats
    stats = test_client.get_system_stats()
    
    print("\n✅ ComfyUI accessible!")
    
    if stats:
        system = stats.get('system', {})
        print(f"\n📊 Statistiques Système:")
        print(f"   - PyTorch: {system.get('pytorch_version', 'N/A')}")
        print(f"   - CUDA: {system.get('cuda', 'N/A')}")
        print(f"   - ComfyUI: {system.get('comfyui_version', 'N/A')}")
        print(f"   - Python: {system.get('python_version', 'N/A')}")
    
except Exception as e:
    print("\n❌ ComfyUI non accessible!")
    print(f"   Erreur: {e}")
    print("   Vérifier que le service est démarré sur port 8188")

## Cell 3: Créer Client ComfyUI

In [ ]:
# Créer client
try:
    client = create_client()
    print("✅ Client ComfyUI créé avec succès")
except ConnectionError as e:
    print(f"❌ Erreur connexion: {e}")
    client = None

## Cell 4: Génération Text-to-Image Simple

In [ ]:
# Génération image test
if client:
    print("🎨 Lancement génération...")
    print("   Prompt: 'A beautiful sunset over mountains'")
    print("   Résolution: 512x512")
    print("   Steps: 20")
    print("\n⏳ Génération en cours (5-10 secondes)...\n")
    
    prompt_id = client.generate_text2image(
        prompt="A beautiful sunset over mountains",
        negative_prompt="blurry, low quality, distorted",
        width=512,
        height=512,
        steps=20,
        cfg=7.0,
        seed=42,  # Reproductible
        save_prefix="test_phase13a"
    )
    
    if prompt_id:
        print(f"\n✅ Génération réussie!")
        print(f"   Prompt ID: {prompt_id}")
        print(f"\n💾 Image sauvegardée dans ComfyUI output/")
    else:
        print("\n❌ Génération échouée")
        print("   Vérifier logs ComfyUI pour détails")
else:
    print("❌ Client non initialisé")

## Cell 5: Résumé Test

In [ ]:
# Résumé
print("="*60)
print("📋 RÉSUMÉ TEST PHASE 13A")
print("="*60)

if client and prompt_id:
    print("\n✅ Status: SUCCESS")
    print("\n🎯 Accomplissements:")
    print("   ✅ ComfyUI accessible")
    print("   ✅ Client Python opérationnel")
    print("   ✅ Génération text-to-image validée")
    print("   ✅ Bridge ComfyUI fonctionnel")
    
    print("\n🚀 Prêt pour Phase 13B:")
    print("   - Implémentation notebook 02-1 complet")
    print("   - Workflows avancés (image-to-image, etc.)")
    print("   - Intégration pédagogique complète")
else:
    print("\n⚠️ Status: PARTIAL")
    print("\n❌ Problèmes détectés:")
    if not client:
        print("   - Connexion ComfyUI échouée")
    if client and not prompt_id:
        print("   - Génération échouée (workflow incompatible?)")
    
    print("\n📚 Troubleshooting:")
    print("   1. Vérifier ComfyUI démarré: http://localhost:8188")
    print("   2. Vérifier modèle Qwen chargé")
    print("   3. Consulter logs ComfyUI")
    print("   4. Voir: TROUBLESHOOTING.md")

print("\n" + "="*60)